# Stop Sign Detection

In [2]:
# imports
import numpy as np
import cv2
import os
import math

## Data Set Folder

In [3]:
image_path = "./archive/images"

## Test Result

In [8]:
#blurring and smoothin
img1=cv2.imread(os.path.join(image_path,'road89.png'),1)

hsv = cv2.cvtColor(img1,cv2.COLOR_BGR2HSV)

originalImage = img1
img1 = removeShadows(img1)
#lower red
lower_red = np.array([0,50,50])
upper_red = np.array([10,255,255])


#upper red
lower_red2 = np.array([170,50,50])
upper_red2 = np.array([180,255,255])

mask = cv2.inRange(hsv, lower_red, upper_red)
res = cv2.bitwise_and(img1,img1, mask= mask)


mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
res2 = cv2.bitwise_and(img1,img1, mask= mask2)

img3 = res+res2
img4 = cv2.add(res,res2)
img5 = cv2.addWeighted(res,0.5,res2,0.5,0)


kernel = np.ones((15,15),np.float32)/225
smoothed = cv2.filter2D(res,-1,kernel)
smoothed2 = cv2.filter2D(img3,-1,kernel)

cv2.imshow('Original',img1)
cv2.imshow('Averaging',smoothed)
cv2.imshow('mask',mask)
cv2.imshow('res',res)
cv2.imshow('mask2',mask2)
cv2.imshow('res2',res2)
cv2.imshow('res3',img3)
cv2.imshow('res4',img4)
cv2.imshow('res5',img5)
cv2.imshow('smooth2',smoothed2)




cv2.waitKey(0)
cv2.destroyAllWindows() 

In [6]:

def closing(testImgRed):
    kernel = np.ones((3,3),np.uint8)
    testImgRed = cv2.morphologyEx(testImgRed, cv2.MORPH_CLOSE, kernel)
    return testImgRed

def removeShadows(testImg):

    rgb_planes = cv2.split(testImg)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)

    testImg = cv2.merge(result_norm_planes)

    return testImg

img = img5
#cv2.imshow('img1',img[:,:,0])

#img = closing(img)

ret,thresh1 = cv2.threshold(img[:,:,0], 0, 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
thresh1 = cv2.bitwise_not(thresh1)

# Creating kernel
kernel = np.ones((5, 5), np.uint8)

kernel2 = np.ones((3, 3), np.uint8)
#thresh1 = cv2.erode(thresh1, kernel2) 
#thresh1 = cv2.dilate(thresh1, kernel)
# Using cv2.erode() method 





#thresh1 = closing(thresh1)

cv2.imshow('thresh', thresh1)       
cv2.waitKey(0)
cv2.destroyAllWindows()

contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
newImage = originalImage.copy()
for cnt in contours:
    approx = cv2.approxPolyDP(cnt,0.0125*cv2.arcLength(cnt,True),True)
    if len(approx)==8:
        cv2.drawContours(newImage, [cnt], 0, (0, 255, 0), 6)

cv2.imshow('sign', newImage)       
cv2.waitKey(0)
cv2.destroyAllWindows()